In [1]:
import numpy as np
import os

from neuron import h
from neuron.units import mV, ms
h.load_file("stdrun.hoc")
h.load_file("stdlib.hoc")
h.load_file("import3d.hoc")

import sys
sys.path.insert(1, "../../utils/")
import Stimuli, Pyr, HH

In [2]:
duration = 1000

In [3]:
hh = HH.HH()

In [4]:
def psection(sec):
    from neuron import h

    try:
        from neuron import rxd
        from neuron.rxd import region, species
        from neuron.rxd import rxd as rxd_module

        have_rxd = True
    except:
        have_rxd = False

    results = {}

    mname = h.ref("")

    # point processes
    pps = {}
    mt = h.MechanismType(1)
    for i in range(int(mt.count())):
        mt.select(i)
        mypps = set()
        pp = mt.pp_begin(sec=sec)
        while pp is not None:
            mypps.add(pp)
            pp = mt.pp_next()
        if mypps:
            mt.selected(mname)
            pps[mname[0]] = mypps
    results["point_processes"] = pps

    center_seg_dir = dir(sec(0.5))

    mechs_present = []

    # membrane mechanisms
    mt = h.MechanismType(0)

    for i in range(int(mt.count())):
        mt.select(i)
        mt.selected(mname)
        name = mname[0]
        if name in center_seg_dir:
            mechs_present.append(name)

    results["density_mechs"] = {}

    for mech in mechs_present:
        my_results = {}
        ms = h.MechanismStandard(mech, 3)
        for j in range(int(ms.count())):
            n = int(ms.name(mname, j))
            name = mname[0]
            pvals = []
            # TODO: technically this is assuming everything that ends with _ion
            #       is an ion. Check this.
            if mech.endswith("_ion"):
                pvals = [getattr(seg, name) for seg in sec]
            else:
                mechname = name  # + '_' + mech
                for seg in sec:
                    if n > 1:
                        pvals.append([getattr(seg, mechname)[i] for i in range(n)])
                    else:
                        pvals.append(getattr(seg, mechname))
            my_results[
                name[: -(len(mech) + 1)] if name.endswith("_" + mech) else name
            ] = pvals
        # TODO: should be a better way of testing if an ion
        if not mech.endswith("_ion"):
            results["density_mechs"][mech] = my_results

    results["v"] = [seg.v for seg in sec]

    return results

In [5]:
hh.axon.nseg = 3
psection(hh.axon)

{'point_processes': {},
 'density_mechs': {'hh': {'m': [0.0, 0.0, 0.0],
   'h': [0.0, 0.0, 0.0],
   'n': [0.0, 0.0, 0.0]}},
 'v': [-65.0, -65.0, -65.0]}

In [6]:
h.finitialize(-65)

1.0

In [7]:
# load stimuli
stim_locs_file = '../../data/morpho/segment_arrays/segment_array_8.2Hz_ID16.npy'

stim_params = Stimuli.ExperimentalStimParams()
stim_scaffold = stim_params.stim_scaffold['pyr']

stim_locs = np.load(stim_locs_file)

stimuli = Stimuli.MorphoStimuli(
    f'stimset',
    stim_scaffold['stim_type_array'],
    stim_locs, stim_scaffold,
    duration
)

In [8]:
pyr = Pyr.Pyr(recording=True)

In [9]:
pyr.all_state_vars()

{'Morse_et_al_2010_instance.soma': {'point_processes': {},
  'density_mechs': {'pas': {},
   'cacum': {},
   'cagk': {'o': [0.0]},
   'cal': {'m': [0.0]},
   'can': {'m': [0.0], 'h': [0.0]},
   'cat': {'m': [0.0], 'h': [0.0]},
   'ds': {},
   'hd': {'l': [0.0]},
   'kap': {'n': [0.0], 'l': [0.0]},
   'kdr': {'n': [0.0]},
   'na3': {'m': [0.0], 'h': [0.0], 's': [0.0]}},
  'v': [-65.0]},
 'Morse_et_al_2010_instance.axon': {'point_processes': {},
  'density_mechs': {'pas': {},
   'kdr': {'n': [0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0]},
   'nax': {'m': [0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0],
    'h': [0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,
     0.0,

In [13]:
type(print_state(pyr))

100.00000000001417:-62.17163140293694


NoneType

In [14]:
duration = 100
n = 50
def print_state(pyr):
    print(f'{h.t}:{pyr.cell.soma.v}')
    
h.celsius = 35
h.finitialize()

histories_times = np.sort(np.random.uniform(0, duration, n))
cvode_events = []
for i, history_time in enumerate(histories_times):
    cvode_events.append(h.CVode().event(history_time, lambda: print_state(pyr)))
    
h.continuerun(duration)

3.000000000000009:-62.17536300086177
7.32500000000007:-62.17723927158039
9.000000000000023:-62.17746152945221
11.924999999999857:-62.177609273059375
12.199999999999841:-62.17761392794001
12.349999999999833:-62.17761601427109
15.124999999999675:-62.177613695926674
15.249999999999668:-62.177612324902256
17.999999999999513:-62.17756954428983
18.149999999999505:-62.177566797575324
24.22499999999916:-62.17745754431254
25.924999999999063:-62.17743233128863
26.34999999999904:-62.17742652319333
29.524999999998858:-62.177389341800065
31.299999999998757:-62.177373076640265
31.77499999999873:-62.17736923174465
32.84999999999891:-62.17736127534098
33.449999999999044:-62.17735726546588
36.99999999999985:-62.17733920178061
45.45000000000177:-62.177325477472785
47.3250000000022:-62.17732632173569
48.800000000002534:-62.17732768685813
50.22500000000286:-62.177329512985
53.75000000000366:-62.177335782333444
54.300000000003784:-62.177336947605646
56.725000000004336:-62.17734256193068
57.80000000000458:-

0.0

In [11]:
list(pyr._vs[0])

[-65.0,
 -64.99211647749712,
 -64.98349023436774,
 -64.97381047214166,
 -64.96299907464893,
 -64.9510578990126,
 -64.9380198903243,
 -64.92393038263069,
 -64.9088393077324,
 -64.89279772278496,
 -64.8758561882799,
 -64.85806399912111,
 -64.83946883346503,
 -64.82011661459018,
 -64.80005148322591,
 -64.77931582650625,
 -64.75795033447663,
 -64.73599406828139,
 -64.71348453143115,
 -64.6904577396324,
 -64.6669482869708,
 -64.64298940753952,
 -64.61861303232153,
 -64.59384984151454,
 -64.5687293126716,
 -64.54327976510343,
 -64.51752840099836,
 -64.49150134369346,
 -64.46522367349095,
 -64.43871946136909,
 -64.41201180089043,
 -64.38512283856724,
 -64.35807380290352,
 -64.33088503229811,
 -64.30357600196183,
 -64.27616534997522,
 -64.24867090259015,
 -64.2211096988594,
 -64.19349801466132,
 -64.16585138617359,
 -64.13818463283812,
 -64.1105118798497,
 -64.08284658019338,
 -64.05520153624917,
 -64.02758892097724,
 -64.00002029869296,
 -63.97250664543782,
 -63.94505836894971,
 -63.917685328